# Modelo de predicción de desplazamientos desde departamentos

Con este modelo se pretende predecir el **número de movimientos de salida** con origen en un departamento determinado. Esto ha sido posible gracias a los datos suministrados por **LUCA**, una unidad de servicios Big Data de Telefónica:

[Web de LUCA](https://www.luca-d3.com/)

Cargamos las **librerias** necesarias

In [19]:
library(mlbench)
library(caret)
library(randomForest)
library(Hmisc)
library(h2o)

Iniciamos **H2O**

In [20]:
h2o <- h2o.init()

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         23 minutes 41 seconds 
    H2O cluster version:        3.10.3.6 
    H2O cluster version age:    1 month and 27 days  
    H2O cluster name:           H2O_started_from_R_Ordenador_njn934 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   1.20 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.2 (2016-10-31) 



Cargamos los datos (Datos cerrados, no es posible compartirlos)

In [21]:
journeys <- read.csv2(".../hack4good_journeys.csv", sep = ',')
mun_dep <- read.csv2(".../CO_ids_nombres.csv", sep = ';')

In [22]:
head(journeys)

date_dt,cod_mpio_origin,cod_mpio_destination,journeys,people
2015-02-01,2,2,109,101
2015-02-01,2,6,53,53
2015-02-01,3,3,319,289
2015-02-01,4,4,18345,17653
2015-02-01,4,13,35,35
2015-02-01,4,40,34,34


In [23]:
head(mun_dep)

id_mpio,nombre_mpio,id_dpto,nombre_dpto
1083,EL ENCANTO,91,AMAZONAS
1081,LA CHORRERA,91,AMAZONAS
1082,LA PEDRERA,91,AMAZONAS
1101,LA VICTORIA,91,AMAZONAS
1085,LETICIA,91,AMAZONAS
1078,MIRITI-PARANA,91,AMAZONAS


## Random forest

Este algoritmo genera una gran cantidad de **decision trees** individuales, cada uno de estos formados con una muestra aleatoria de variables y observaciones del data set. Cada decision tree genera independientemente su predicción y el algoritmo en su conjunto se queda con aquella predicción más elegida, teniendo en cuenta así distintas agrupaciones posibles de variables y de individuos.

Se ha realizado un estudio estadístico/matemático de los datos y se ha apreciado un **patrón estacional** en los desplazamientos entre departamentos. Se ha conseguido introducir este patrón gracias a la inclusión de **retardos** en los movimientos entre departamentos. 

In [24]:
model_pred_to_dest <- function(dpto)
{
  # ID de los departamentos a predecir
  id <- unique(mun_dep$id_dpto[which(mun_dep$nombre_dpto==dpto)])
  
  # Seleccionamos los departamentos de destino de los departamentos a predecir
  journeys_slc <- journeys[which(journeys$cod_mpio_origin==id),] #
  
  if (length(journeys_slc$date_dt) > 0){
    # Formato de las fechas
    journeys_slc$date_dt <- as.Date(journeys_slc$date_dt)
    
    # División en training y test sets de acuerdo a la fecha
    test <- journeys_slc[which(journeys_slc$date_dt>='2015-04-15'), ]
    train <- journeys_slc[which(journeys_slc$date_dt<'2015-04-15'), ]
    train <- train[order(train[,3]), ]
    test <- test[order(test[,3]), ]
    cod_mpio_destinations <- unique(train$cod_mpio_origin)
    
    # Inclusión de retardos en los desplazamientos en el training set
    TRAIN <- list()
    
    for (j in 1:length(cod_mpio_destinations)){
      if (cod_mpio_destinations[j] %in% train$cod_mpio_origin){
        train_dest <- train[which(train$cod_mpio_origin == cod_mpio_destinations[j]), ]
        for (i in 1:7){
          colname = paste('lag',i)
          train_dest[colname] <- NA
          train_dest[colname] <- Lag(train_dest$journeys, i)
          train_dest[colname] <- NA
          train_dest[colname] <- Lag(train_dest$journeys, i)
        }
      }
      TRAIN[[j]] <- train_dest
    }
    
    # Inclusión de retardos en los desplazamientos en el test set
    TEST <- list()
    
    for (j in 1:length(cod_mpio_destinations)){
      if (cod_mpio_destinations[j] %in% test$cod_mpio_origin){
        test_dest <- test[which(test$cod_mpio_origin == cod_mpio_destinations[j]),]
        for (i in 1:7){
          colname = paste('lag',i)
          test_dest[colname] <- NA
          test_dest[colname] <- Lag(test_dest$journeys, i)
          test_dest[colname] <- NA
          test_dest[colname] <- Lag(test_dest$journeys, i)
        }
      }
      TEST[[j]] <- test_dest
    }
    
    # Combinamos el TRAIN  y TEST
    train <- do.call(rbind.data.frame, TRAIN)
    test <- do.call(rbind.data.frame, TEST)
    
    # Seleccionamos las variables independientes y dependientes
    predictors <- colnames(train[, !(names(train) %in% c('journeys','people','cod_mpio_destination'))])
    target <- 'journeys'
    
    # Creamos un training y un test set del training set original
    train.train <- train[1:round(nrow(train)*0.8), ]
    train.test <- train[round(nrow(train)*0.8):nrow(train), ]
    
    # Cargamos estos datos en la nube de H2O
    train.train.frame <- as.h2o(train.train, cod_mpio_destination_frame = "train.train")
    train.test.frame <- as.h2o(train.test, cod_mpio_destination_frame = "train.test")
    
    # Random Forest sobre el train.train set
    rf.train = h2o.randomForest(
      x = predictors,
      y = target,
      training_frame = train.train.frame,
      validation_frame = NULL,
      model_id = "rf_train",
      ntrees = 500, mtries = 8)
    
    # Predicciones y residuales
    test.pred <- as.data.frame(h2o.predict(rf.train, train.test.frame))
    resid.test <- train.test$journeys - test.pred$predict
    
    # Cargamos los datos del training set completo en la nube de H2O
    train.frame <- as.h2o(train, cod_mpio_destination_frame = "train")
    
    # Random Forest sobre el training set completo
    rf.trainf = h2o.randomForest(
      x = predictors,
      y = target,
      training_frame = train.frame,
      validation_frame = NULL,
      model_id = "rf_train_tot",
      ntrees = 500, mtries = 6)
    
    # Cargamos los datos del test set completo en la nube de H2O
    test.frame <- as.h2o(test, cod_mpio_destination_frame = "test")
    
    # Predicciones y resultados finales
    predictions.rf <- as.data.frame(h2o.predict(rf.trainf, test.frame))
    resid.test <- test$journeys - predictions.rf$predict
    
    FINAL <- list(depart = dpto, mae = mean(abs(resid.test)))
  }
  else{ FINAL <- list()}
  
}

Seleccionamos el departamento o departamentos a predecir

In [27]:
unique(mun_dep$nombre_dpto)

[1] AMAZONAS                                               
 [2] ANTIOQUIA                                              
 [3] ARAUCA                                                 
 [4] ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA
 [5] ATLANTICO                                              
 [6] BOLIVAR                                                
 [7] BOYACA                                                 
 [8] CALDAS                                                 
 [9] CAQUETA                                                
[10] CASANARE                                               
[11] CAUCA                                                  
[12] CESAR                                                  
[13] CHOCO                                                  
[14] CORDOBA                                                
[15] CUNDINAMARCA                                           
[16] GUAINIA                                                
[17] GUAVIARE                                               
[18] HUILA                                                  
[19] LA GUAJIRA                                             
[20] MAGDALENA                                              
[21] META                                                   
[22] NARIÃO                                                
[23] NORTE DE SANTANDER                                     
[24] PUTUMAYO                                               
[25] QUINDIO                                                
[26] RISARALDA                                              
[27] SANTAFE DE BOGOTA D.C                                  
[28] SANTANDER                                              
[29] SUCRE                                                  
[30] TOLIMA                                                 
[31] VALLE DEL CAUCA                                        
[32] VAUPES                                                 
[33] VICHADA                                                
33 Levels: AMAZONAS ANTIOQUIA ... VICHADA

Nosotros seleccionaremos a **Antioquía** y **Vaupes**

In [30]:
DEPT <- unique(mun_dep$nombre_dpto)[c(2, 32)]
DEPT

[1] ANTIOQUIA VAUPES   
33 Levels: AMAZONAS ANTIOQUIA ... VICHADA

A continuación se realiza la predicción con la siguiente función

In [34]:
RESULTS <- list()
for (i in 1:length(DEPT)) {
  dept <- DEPT[i]
  RESULTS[[i]] <- model_pred_to_dest(dept)
}

  |======================================================================| 100%
  |======================================================================| 100%


Warning message in .h2o.startModelJob(algo, params, h2oRestApiVersion):
"Dropping constant columns: [cod_mpio_origin].
"

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


Warning message in .h2o.startModelJob(algo, params, h2oRestApiVersion):
"Dropping constant columns: [cod_mpio_origin].
"

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


Warning message in .h2o.startModelJob(algo, params, h2oRestApiVersion):
"Dropping constant columns: [cod_mpio_origin].
"

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


Warning message in .h2o.startModelJob(algo, params, h2oRestApiVersion):
"Dropping constant columns: [cod_mpio_origin].
"

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


Y mostramos los resultados

In [35]:
RESULTS

[[1]]
[[1]]$depart
[1] ANTIOQUIA
33 Levels: AMAZONAS ANTIOQUIA ... VICHADA

[[1]]$mae
[1] 15.2945


[[2]]
[[2]]$depart
[1] VAUPES
33 Levels: AMAZONAS ANTIOQUIA ... VICHADA

[[2]]$mae
[1] 3.093841

Utilizando el **MAE** (*mean absolute error*) como medida de precisión del modelo